In [34]:
import pandas as pd
import math
from scipy import stats
from numpy import random
from numpy import *

In [2]:
booking = pd.read_csv('AB_test_data.csv')
booking.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [3]:
matrix = booking.groupby(['Variant','purchase_TF']).size().reset_index()
matrix.rename(columns={matrix.columns[2]:'num'}, inplace = True)
matrix

,Variant,purchase_TF,num
0,A,False,106298
1,A,True,18702
2,B,False,4117
3,B,True,883


In [4]:
Pa = matrix.loc[1, 'num']/(matrix.loc[0, 'num']+matrix.loc[1, 'num'])
print('Pa: ', Pa)
Pb = matrix.loc[3, 'num']/(matrix.loc[2, 'num']+matrix.loc[3, 'num'])
print('Pb: ', Pb)
P = (Pa+Pb)/2
print('P: ', P)
n = matrix.loc[2, 'num'] + matrix.loc[3, 'num']
print('n: ',n)
ZScore = (Pb-Pa)/math.sqrt(Pa*(1-Pa)/n)
print('ZScore: ',ZScore)

Pa:  0.149616
Pb:  0.1766
P:  0.163108
n:  5000
ZScore:  5.349273094732516


In [5]:
z = stats.norm.ppf(0.99)# 置信度为95%的置信区间
z

2.3263478740408408

# problem 2

In [6]:
Ta = stats.norm.ppf(0.975)
print('tα/2: ', Ta)
Tb = stats.norm.ppf(0.2)
print('tβ: ', Tb)
#Tb = stats.norm.interval(0.95,loc=0,scale=1)

tα/2:  1.959963984540054
tβ:  -0.8416212335729142


In [7]:
c1 = math.sqrt(2*P*(1-P))
print('c1: ', c1)
c2 = math.sqrt(Pa*(1-Pa) + Pb*(1-Pb))
print('c2: ', c2)

c1:  0.5225012542300735
c2:  0.5221527482873187


In [8]:
print(P)
print()

0.163108



In [9]:
nOptim = (Ta*c1-Tb*c2)**2 * (1/(Pa-Pb)**2)
nOptim

2941.6814032458096

In [10]:
matrix

,Variant,purchase_TF,num
0,A,False,106298
1,A,True,18702
2,B,False,4117
3,B,True,883


In [25]:
def OptiNum (list1):
    matrix = list1.groupby(['Variant','purchase_TF']).size().reset_index()
    matrix.rename(columns={matrix.columns[2]:'num'}, inplace = True)
    
    Pa = matrix.loc[1, 'num']/(matrix.loc[0, 'num']+matrix.loc[1, 'num'])
    Pb = matrix.loc[3, 'num']/(matrix.loc[2, 'num']+matrix.loc[3, 'num'])
    P = (Pa+Pb)/2
    
    Ta = stats.norm.ppf(0.975)
    Tb = stats.norm.ppf(0.2)
    
    c1 = math.sqrt(2*P*(1-P))
    c2 = math.sqrt(Pa*(1-Pa) + Pb*(1-Pb))
    
    nOptim = (Ta*c1-Tb*c2)**2 * (1/(Pa-Pb)**2)
    
    return nOptim

In [26]:
def ztest (list1,n,pa):
    matrix = list1.groupby(['Variant','purchase_TF']).size().reset_index()
    matrix.rename(columns={matrix.columns[2]:'num'}, inplace = True)
    Pa = pa
    Pb = matrix.loc[1, 'num']/(matrix.loc[0, 'num']+matrix.loc[1, 'num'])
    P = (Pa+Pb)/2
    ZScore = (Pb-Pa)/math.sqrt(Pb*(1-Pb)/n)
    return ZScore

In [27]:
df1 = []
for i in range(0,10):
    booking1 = booking[booking['Variant']=='B']
    booking2 = booking1.sample(2942)
    zscore = ztest(booking2,2942,0.149616)
    df1.append(zscore)
    
df1

[3.538812740995572,
 4.129692674930797,
 4.039481276467514,
 3.6761276488534276,
 3.4469420232005343,
 3.3086381012358568,
 3.949021582642977,
 3.2624026806335147,
 3.400907432262858,
 3.7673473082281697]

In [54]:
#A = (1-0.2)/0.05
#B = 0.2/(1-0.05)
A = 20
B = 0.2
Pa = 0.149616

#Pbn = []
n = []
n2 = []
for i in range(10):
    booking1 = booking[booking['Variant']=='B']
    booking2 = booking1.sample(2942)
    matrix = booking2.groupby(['Variant','purchase_TF']).size().reset_index()
    matrix.rename(columns={matrix.columns[2]:'num'}, inplace = True)
    Pb = matrix.loc[1, 'num']/(matrix.loc[0, 'num']+matrix.loc[1, 'num'])
    c=0
    a = 1
    for j in booking2['purchase_TF']:
        c+=1
        if j == True:
            l = Pb / Pa
        else:
            l = (1-Pb)/(1-Pa)
        a = a * l
        if a<=B or a>=A:
            n.append(c)
            if a<=B:
                n2.append(c)
            break
n

[201, 312, 400, 962, 811, 658, 2025, 350, 176, 627]

In [55]:
n2

[658, 350, 627]

In [56]:
mean(n)

652.2